Installs:

In [ ]:
! pip install emoji
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.5 MB/s eta 0:00:00


Imports:

In [ ]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel


# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, precision_recall_fscore_support

# Data and preprocessing
import pandas as pd
import numpy as np
import re
import html
import emoji


# Optimization & logging
import optuna
import wandb

#collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required packages
!pip install nlpaug transformers

import nlpaug.augmenter.word as naw
import random
from tqdm import tqdm
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid tokenizer warnings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 27.9 MB/s eta 0:00:00


In [ ]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
wandb.login()

Using device: cuda


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yardenr1 (yardenr1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

---

f423cfbdb50d571d7922fe745356685617100809

Load data:

In [ ]:
# === Load data ===
train_df = pd.read_csv('/content/drive/MyDrive/deep_learning/train_processed.csv', encoding='latin1')
eval_df = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv', encoding='latin1')
test_df = pd.read_csv('/content/drive/MyDrive/deep_learning/test_processed.csv', encoding='latin1')

# Create label mappings
ordered_labels = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
label2id = {label: i for i, label in enumerate(ordered_labels)}
id2label = {i: label for label, i in label2id.items()}

train_df["label"] = train_df["Sentiment"].map(label2id)
eval_df["label"] = eval_df["Sentiment"].map(label2id)
test_df["label"] = test_df["Sentiment"].map(label2id)


In [ ]:
# Display
pd.set_option('display.max_colwidth', None)  # so full text is shown
sample = train_df[['ProcessedTweet','Sentiment','label']].sample(10, random_state=24)
display(sample)

,ProcessedTweet,Sentiment,label
17977,new survey by user finds that only 2% of #consumers think #brands should pause all #advertising; 49% want #ads to make them feel informed and 37% want ads to make them feel warm/happy #digitalmarketing #covid #consumerbehavior http,Extremely Positive,4
31793,some #costco stores say their prohibiting people from returning #toiletpaper. who returns toilet paper? probably the same people who think their rectums go into remission? #trending #covid user #jimmy #kimmel #for #real,Neutral,2
8808,"after increasing the prices of platform tickets, the indian railways withdrew most concessional ticket facilities in trains as a precautionary measure to contain the spread of #covid. http",Neutral,2
23027,"user money hdelacerda. oil prices caused a massive slow down and now the covid 19 has caused layoffs, i havent worked since the beginning of february just trying to make sure my wife and 2 daughters are taken care of.",Extremely Positive,4
6409,my wife just messaged me to say the supermarket shelves were empty. i know what's for dinner tonight #covid http,Negative,1
28137,"i nominate the global coronavirus responders for the 2020 time magazines person of the year: (including nurses, paramedics, doctors, hospital admin and cleaning staff, grocery store employees, etc) #covid",Neutral,2
22277,"wed like to take a moment to thank all of the grocery store personnel, cleaners, maintenance workers, truck drivers and warehouse workers for their perseverance and commitment. we are incredibly grateful to those who are making sacrifices to support the covid 19 fight. http",Extremely Positive,4
21206,this is too much! no fizzy san pelegrno water for my wine spritzer or houmous! what in gods name do you expect the middle classes to do! to die like dinosaurs! #covid #stay #home #24 #in #48 #stockpiling #uk #toiletpaper #covid,Extremely Negative,0
15208,? money cag money gis money k money ko consumer brands association ceo on the state of global supply chains amid covid 19 http,Neutral,2
28598,breaking: govt. has announced good news for shoppers who have been battling to buy essential products at inflated prices. retailers on the naughty list could face hefty fines or even prison sentences. #covid http,Negative,1


---

Dataset class:

In [ ]:
class CoronaTweetDataset(Dataset):
    """
    Dataset class for loading COVID-19 tweets for sentiment classification.
    Compatible with HuggingFace transformers and PyTorch.
    """
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['ProcessedTweet'].tolist()  # Cleaned tweets
        self.labels = dataframe['label'].tolist()          # Encoded sentiment labels
        self.tokenizer = tokenizer                         # Tokenizer for the model

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize input text with padding and truncation
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=128,       # 128 is enough for tweets
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),             # Tensor of token IDs
            'attention_mask': encoding['attention_mask'].squeeze(),   # Tensor mask
            'labels': torch.tensor(label, dtype=torch.long)           # Ground truth label
        }




---



Traning:

In [ ]:
def early_stop_check(patience, best_val_accuracy, best_val_accuracy_epoch, current_val_accuracy, current_val_accuracy_epoch):
    early_stop_flag = False
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy
        best_val_accuracy_epoch = current_val_accuracy_epoch
    else:
        if current_val_accuracy_epoch - best_val_accuracy_epoch > patience:
            early_stop_flag = True
    return best_val_accuracy, best_val_accuracy_epoch, early_stop_flag

In [ ]:
# ============ UPDATED TRAINING FUNCTION WITH GRADIENT CLIPPING ============
def train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs, patience, trial, gradient_clip_val=None):
    """
    Updated training function with gradient clipping and enhanced monitoring
    """
    best_val_accuracy = 0.0
    best_val_accuracy_epoch = 0
    early_stop_flag = False
    best_model_state = None

    for epoch in range(1, epochs + 1):
        # Training phase
        model.train()
        train_loss = 0.0
        total_train_samples = 0
        correct_train_predictions = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            loss.backward()

            # NEW: Gradient Clipping
            if gradient_clip_val is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)

            optimizer.step()

            # Accumulate training metrics
            train_loss += loss.item() * input_ids.size(0)
            total_train_samples += input_ids.size(0)
            correct_train_predictions += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        # Validation phase
        model.eval()
        val_loss = 0.0
        total_val_samples = 0
        correct_val_predictions = 0
        all_val_labels = []
        all_val_preds = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                total_val_samples += input_ids.size(0)
                correct_val_predictions += (logits.argmax(dim=1) == labels).sum().item()

                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(logits.argmax(dim=1).cpu().numpy())

        # Calculate per-class metrics
        class_names = ['Extremely Negative', 'Extremely Positive', 'Negative','Neutral','Positive']
        precisions, recalls, f1s, _ = precision_recall_fscore_support( all_val_labels, all_val_preds, average=None, labels=[0, 1, 2, 3, 4], zero_division=0)

        # Log per-class metrics to wandb
        for i, class_name in enumerate(class_names):
            wandb.log({
                f"Class_{class_name}_Precision": precisions[i] if i < len(precisions) else 0.0,
                f"Class_{class_name}_Recall": recalls[i] if i < len(recalls) else 0.0,
                f"Class_{class_name}_F1": f1s[i] if i < len(f1s) else 0.0
            })

        # Calculate overall metrics
        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples
        val_precision = precision_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_recall = recall_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='macro', zero_division=0)

        # Early stopping check
        best_val_accuracy, best_val_accuracy_epoch, early_stop_flag = early_stop_check(
            patience, best_val_accuracy, best_val_accuracy_epoch, val_accuracy, epoch
        )

        # Save best model
        if val_accuracy == best_val_accuracy:
            best_model_state = model.state_dict()

        # Log metrics to wandb
        wandb.log({
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_accuracy,
            "Validation Precision": val_precision,
            "Validation Recall": val_recall,
            "Validation F1": val_f1
        })

        # Print progress
        print(f"Epoch {epoch:2d}/{epochs}: "
              f"Train Acc: {train_accuracy:.4f}, "
              f"Val Acc: {val_accuracy:.4f}, "
              f"Val F1: {val_f1:.4f}, "
              f"Gap: {(train_accuracy - val_accuracy):.4f}")

        if len(f1s) == 3:
            print(f"          Per-class F1: "
                  f"Neg: {f1s[0]:.3f}, "
                  f"Neu: {f1s[1]:.3f}, "
                  f"Pos: {f1s[2]:.3f}")

        if early_stop_flag:
            print(f"Early stopping at epoch {epoch}")
            break

    if best_model_state is not None:
        torch.save(best_model_state, f"best_model_trial_{trial.number}.pt")

    return best_val_accuracy


In [ ]:
# ============ CLASS WEIGHTS COMPUTATION ============
from sklearn.utils.class_weight import compute_class_weight
def compute_class_weights(train_labels):
    """
    Compute class weights for your imbalanced dataset
    """
    train_labels = np.array(train_labels)
    unique_labels = np.unique(train_labels)

    print(f"Computing class weights for {len(train_labels)} samples")
    print(f"Label distribution: {np.bincount(train_labels)}")

    class_weights = compute_class_weight(
        'balanced',
        classes=unique_labels,
        y=train_labels
    )

    print(f"Class weights computed:")
    for i, label in enumerate(unique_labels):
        label_name = ['Extremely Negative', 'Extremely Positive', 'Negative','Neutral','Positive'][label]
        print(f"{label_name} ({label}): {class_weights[i]:.3f}")

    return torch.tensor(class_weights, dtype=torch.float)

#=============== manual class weight=====================
#def compute_class_weights(train_labels):
   #Weighting more for harder classes
 # manual_weights = torch.tensor([
  #  1.0,  # Extremely Negative – no recall issue
  #  1.0,  # Negative – balanced
  #  1.2,  # Neutral – low recall (0.81)
  #  1.0,  # Positive – high recall (0.91)
   # 1.3   # Extremely Positive – lowest recall (0.80
    #  ], dtype=torch.float).to(device)
 # return torch.tensor(manual_weights, dtype=torch.float)



# ============ CUSTOM ROBERTA WITH DROPOUT ============
class RobertaWithDropout(nn.Module):
    """
    Custom RoBERTa model with configurable dropout
    """
    def __init__(self, model_name, num_labels, dropout_rate=0.1):
        super(RobertaWithDropout, self).__init__()

        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return type('obj', (object,), {'logits': logits})()


Zero study hippper parameters:

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 2e-6
learning_rate_to = 5e-5
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [5, 7]
batch_size_choices = [32, 64, 128]
num_layers_choices = [0,1]
dropout_rate_choices =  [0.1, 0.2, 0.3, 0.4]
gradient_clip_val_choices = [0.5, 1.0, 2.0]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True


First study hippper parameters:

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 1e-4
learning_rate_to = 3e-4
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [5, 7]
batch_size_choices = [64, 128]
num_layers_choices = [3, 4, 5]
dropout_rate_choices = [0.2, 0.25]
gradient_clip_val_choices = [0.5, 1.0, 1.5]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = False


Seconed study hippper parameters:

In [ ]:
# Focused Search Configuration: Based ONLY on Experiment 0 insights
learning_rate_from = 5e-6                # Narrow range around good low LR
learning_rate_to = 3e-5
weight_decay_from = 1e-6                 # Same as before, works well
weight_decay_to = 5e-5

batch_size_choices = [32, 64]            # Batch size 32 gave best generalization
dropout_rate_choices = [0.1, 0.2]        # 0.1–0.2 showed stability
num_layers_choices = [0]              # Stick to shallow setup
gradient_clip_val_choices = [0.5, 1.0, 1.5]
patience_choices = [5, 7]                # Let Optuna decide stopping horizon

use_class_weights_choices = [True, False]     # Still worth comparing
enable_class_weights = True
enable_gradient_clipping = True

Third + Fourth study hippper parameters:

In [ ]:
learning_rate_from = 2e-6
learning_rate_to = 5e-5
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [7]
batch_size_choices = [32]

num_layers_choices = [0]
dropout_rate_choices =  [0.1, 0.4]
gradient_clip_val_choices = [0.5, 1.0]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True


Fifth studt hupper parameters:

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 2e-6
learning_rate_to = 5e-5
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [5, 7]
batch_size_choices = [32, 64, 128]
num_layers_choices = [0,1,2]
dropout_rate_choices =  [0.1, 0.2, 0.3, 0.4]
gradient_clip_val_choices = [0.5, 1.0, 2.0]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True

Sixth study hipper paramaters:

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 1.8e-5
learning_rate_to   = 2.1e-5
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [5]
batch_size_choices = [32]
num_layers_choices = [0]
dropout_rate_choices =  [ 0.2, 0.3]
gradient_clip_val_choices = [0.5, 1.0]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True

OBJECTIVE:

In [ ]:
# ============ OBJECTIVE FUNCTION ============

def objective(trial,
              learning_rate_from=learning_rate_from, learning_rate_to=learning_rate_to,
              weight_decay_from=weight_decay_from, weight_decay_to=weight_decay_to,
              patience_choices=patience_choices, batch_size_choices=batch_size_choices,
              num_layers_choices=num_layers_choices, dropout_rate_choices=dropout_rate_choices,
              gradient_clip_val_choices=gradient_clip_val_choices, use_class_weights_choices=use_class_weights_choices,
              enable_gradient_clipping=enable_gradient_clipping, enable_class_weights=enable_class_weights):

    # Suggest hyperparameters using the ranges passed in
    learning_rate = trial.suggest_float("learning_rate", learning_rate_from, learning_rate_to, log=True)
    weight_decay = trial.suggest_float("weight_decay", weight_decay_from, weight_decay_to, log=True)
    patience = trial.suggest_categorical("patience", patience_choices)
    #patience =7
    batch_size = trial.suggest_categorical("batch_size", batch_size_choices)
    num_layers = trial.suggest_categorical("num_layers", num_layers_choices)
    dropout_rate = trial.suggest_categorical("dropout_rate", dropout_rate_choices)

    # Optional parameters
    if enable_gradient_clipping:
        gradient_clip_val = trial.suggest_categorical("gradient_clip_val", gradient_clip_val_choices)
    else:
        gradient_clip_val = None

    if enable_class_weights:
        use_class_weights = trial.suggest_categorical("use_class_weights", use_class_weights_choices)
    else:
        use_class_weights = False

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

    # Class weights computation
    class_weights = None
    if use_class_weights:
        print("Computing class weights...")
        train_labels = train_df['label'].values
        class_weights = compute_class_weights(train_labels)

    # Dataset and DataLoader
    train_dataset = CoronaTweetDataset(train_df, tokenizer)
    val_dataset = CoronaTweetDataset(eval_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Use custom model with dropout
    model = RobertaWithDropout(
        "cardiffnlp/twitter-roberta-base-sentiment-latest",
        num_labels=5,
        dropout_rate=dropout_rate
    ).to(device)

    # Freeze/unfreeze layers
    for param in model.roberta.parameters():
        param.requires_grad = False
    for param in model.roberta.encoder.layer[-num_layers:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Use class weights in loss if specified
    if use_class_weights and class_weights is not None:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        print("Using weighted CrossEntropyLoss")
    else:
        criterion = nn.CrossEntropyLoss()
        print("Using standard CrossEntropyLoss")

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Initialize wandb
    wandb.init(
        project="cardiffnlp_new5_6.3",
        config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "dropout_rate": dropout_rate,
            "gradient_clip_val": gradient_clip_val,
            "use_class_weights": use_class_weights,
            "enable_gradient_clipping": enable_gradient_clipping,
            "enable_class_weights": enable_class_weights,
            "architecture": "Twitter-RoBERTa",
            "dataset": "Corona_NLP"
        },
        name=f"trial_{trial.number}"
    )

    # Train with optional gradient clipping
    best_val_accuracy = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion,
        epochs=12, patience=patience, trial=trial,
        gradient_clip_val=gradient_clip_val
    )

    wandb.finish()
    return best_val_accuracy

##Hyperparameter Search Rationale
To systematically explore the effect of different hyperparameter configurations, we split the tuning process into two separate experiments:

### **Experiment 0 — Broad & Conservative Search**
This experiment was designed to cover a wide search space with low model complexity (0–1 added layers), lower learning rates, and more dropout values. The idea was to identify whether a minimal configuration, possibly with strong regularization, could yield a robust baseline.

**Motivation:** This configuration tests robustness under shallow models and regularization techniques. It is especially useful to detect underfitting trends, noisy gradients, or improvements from weighted loss.

### **Experiment 1 — Focused Search on High-Capacity Models**
Here, we narrow the ranges and intentionally increase model complexity (3–5 layers), focusing on fine-tuning deeper classification heads with a tighter learning rate range and less dropout.

**Motivation:** This setting assumes the base encoder requires more capacity at the classifier level. With reduced regularization and stronger classifiers, the goal was to improve validation accuracy and F1 without overfitting.



# **ZERO STUDY:**

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 0:")
study0 = optuna.create_study(direction="maximize")
study0.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study0.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model0.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")

[I 2025-08-03 05:32:27,436] A new study created in memory with name: no-name-d6ca0547-445c-419e-b2ce-99fa3073e483


Starting Study 0:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Extremely Positive (1): 0.830
Negative (2): 1.067
Neutral (3): 0.721
Positive (4): 1.243
Using weighted CrossEntropyLoss


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 15/15: Train Acc: 0.9732, Val Acc: 0.8523, Val F1: 0.8546, Gap: 0.1210


Class_Extremely Negative_F1,▁▆▃▆▇▆▇█▇▆█▇▇█▆
Class_Extremely Negative_Precision,▇▆▁█▅▄▅▆▅▄▆█▅▆▆
Class_Extremely Negative_Recall,▁▅█▃▆▇▆▆▆▇▆▄▆▆▅
Class_Extremely Positive_F1,▁▆▁█▇▅▇█▇▆▇▇▇▅▆
Class_Extremely Positive_Precision,▁▄▄▅▅▃▇▆▇▅▇▆▅█▆
Class_Extremely Positive_Recall,▄▇▁█▇▆▅▇▆▅▆▇▆▃▅
Class_Negative_F1,▃▆████▅██▂█▇▅▁▄
Class_Negative_Precision,▆▇█▆█▇▅▇▇▂▆▅▅▁▄
Class_Negative_Recall,▁▃▄▆▃▄▆▄▅▇▅▆▆█▆
Class_Neutral_F1,▁▄▆▅▆▇▇▅▇▅██▇▇▇
Class_Neutral_Precision,▁▃█▆▆▆▆▇▆█▅▆▆▇▆


[I 2025-08-03 14:48:38,349] Trial 14 finished with value: 0.8674684159378037 and parameters: {'learning_rate': 4.9786903263194e-05, 'weight_decay': 2.269910677069254e-06, 'patience': 7, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.4, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 14 with value: 0.8674684159378037.



 Best Results:
Validation Accuracy: 0.8675
Best hyperparameters:
  learning_rate: 4.9786903263194e-05
  weight_decay: 2.269910677069254e-06
  patience: 7
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.4
  gradient_clip_val: 0.5
  use_class_weights: True

Best model saved: best_model_trial_14.pt


# **FIRST STUDY:**

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 1:")
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study1.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model1.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")

[I 2025-07-30 12:35:49,843] A new study created in memory with name: no-name-5aad679b-fde0-4772-9e21-92fad52b5e0e


Starting Study 1:
Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Extremely Positive (1): 0.830
Negative (2): 1.067
Neutral (3): 0.721
Positive (4): 1.243
Using weighted CrossEntropyLoss


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch  1/15: Train Acc: 0.5279, Val Acc: 0.5866, Val F1: 0.5944, Gap: -0.0587


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 15/15: Train Acc: 0.9427, Val Acc: 0.8203, Val F1: 0.8261, Gap: 0.1224


Class_Extremely Negative_F1,▁▅▄▇▇▇▆▇██▇█▆██
Class_Extremely Negative_Precision,▁▄▃▅▆▅▅▆█▇▅█▄█▇
Class_Extremely Negative_Recall,▆▃▇▇▄▆▇▅▁▄▇▂█▁▄
Class_Extremely Positive_F1,▁▅▅▇▇▇▆▇██▇█▇██
Class_Extremely Positive_Precision,▁▃▅▆▆▃▆▆▅▅█▆▆▆▆
Class_Extremely Positive_Recall,▁▅▄▆▆▇▆▆██▅█▆█▇
Class_Negative_F1,▁▃▆▆▇▇▇▆██▇▇▇▇█
Class_Negative_Precision,▁▆▄▆▇█▅▆▇█▆▅▆▆▇
Class_Negative_Recall,▃▁▇▆▆▅█▆█▇██▇▇▇
Class_Neutral_F1,▁▃▆▆▇█▇▇█▇▇▇███
Class_Neutral_Precision,▁▃▅▆▇▇▇▆▇█▆▇█▇▇


[I 2025-07-30 19:06:55,436] Trial 14 finished with value: 0.8227648202137998 and parameters: {'learning_rate': 0.00013871923622405644, 'weight_decay': 1.106774053285717e-05, 'batch_size': 64, 'num_layers': 5, 'dropout_rate': 0.25, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 13 with value: 0.8341836734693877.


NameError: name 'study' is not defined

---

### **Summary of Results:**

| Metric                          | Experiment 0 | Experiment 1 |
| ------------------------------- | ------------ | ------------ |
| **Validation Accuracy** (mean)  | 76.95%       | **78.61%**   |
| **Validation F1** (mean)        | 77.53%       | **79.26%**   |
| **Train Accuracy** (mean)       | 84.41%       | **92.33%**   |
| **Validation Recall** (mean)    | 78.08%       | **79.74%**   |
| **Validation Precision** (mean) | 77.51%       | **79.26%**   |

Experiment 1 clearly outperforms across all metrics, especially in terms of overfitting reduction and generalization.

---

# **SECONED STUDY:**


Following the results of Experiment 0, we observed that shallow classifier heads (0–1 layers) achieved strong F1 scores across sentiment classes, while maintaining minimal overfitting. Given this, we chose to focus the next hyperparameter search exclusively on shallow architectures, and to refine only the best-performing settings.

The goal of this **targeted run is to isolate the most impactful tuning knobs within the low-complexity regime** — such as dropout regularization, batch size, learning rate, and weight decay — while keeping model depth and clipping values fixed. This allows us to extract maximum performance from the shallow setting before moving on to more complex solutions such as data augmentation or architectural changes.

Run:

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 2:")
study2 = optuna.create_study(direction="maximize")
study2.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study2.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model2.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")



[I 2025-08-03 16:03:14,209] A new study created in memory with name: no-name-6d0daf2d-fccd-4db1-baea-d966e442d1ee


Starting Study 2:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Extremely Positive (1): 0.830
Negative (2): 1.067
Neutral (3): 0.721
Positive (4): 1.243
Using weighted CrossEntropyLoss


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 15/15: Train Acc: 0.9711, Val Acc: 0.8467, Val F1: 0.8501, Gap: 0.1244


Class_Extremely Negative_F1,▁▅▇▆▆▇█▅▇█▇█▇▇█
Class_Extremely Negative_Precision,▁▅▇▆▆▇█▄▇███▇█▇
Class_Extremely Negative_Recall,▆▃▂▅▇▄▃█▆▃▃▅▅▁▅
Class_Extremely Positive_F1,▁▆▇▇▇▇█▆▇███▇██
Class_Extremely Positive_Precision,▁▆▆▆▇▇▇▇▇█▇█▇▇█
Class_Extremely Positive_Recall,▁▅▇▇▆▇█▅▆▇█▇▇▇▇
Class_Negative_F1,▁▆▇▇▇████▇▇▇▇▆▇
Class_Negative_Precision,▂▃▄▇▁█▇▆▅▁▃▄▄▂▁
Class_Negative_Recall,▁▆▆▆▇▆▆▇▇▇▇▆▇▇█
Class_Neutral_F1,▁▆▇▇█▇█▇███████
Class_Neutral_Precision,▁▂▅▇▆▅▆▇▇▇█▅█▇▇


[I 2025-08-04 02:13:41,934] Trial 14 finished with value: 0.8534985422740525 and parameters: {'learning_rate': 1.6368369237343168e-05, 'weight_decay': 3.422900026320533e-06, 'patience': 5, 'batch_size': 64, 'num_layers': 0, 'dropout_rate': 0.1, 'gradient_clip_val': 1.5, 'use_class_weights': True}. Best is trial 10 with value: 0.8671039844509232.



 Best Results:
Validation Accuracy: 0.8671
Best hyperparameters:
  learning_rate: 1.924356648556222e-05
  weight_decay: 4.998886173840331e-06
  patience: 5
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.1
  gradient_clip_val: 0.5
  use_class_weights: False

Best model saved: best_model_trial_10.pt




---



## **Study Results Summary – Stability vs. Peak Performance**

To evaluate the effect of narrowing the hyperparameter space, we compared two experiments:

* **Study 0** – Broad hyperparameter search on shallow models
* **Study 2** – Focused search based on insights from Study 0

The table below presents the results for:

* The **best single run** in Study 0
* The **average performance** of all trials in Study 0 and Study 2

|                          | **Best in Study 0** | **Average Study 0** | **Average Study 2** |
| ------------------------ | ------------------- | ------------------- | ------------------- |
| **Validation Accuracy**  | **86.54%**          | 76.95%              | **84.86%**          |
| **Validation F1**        | **86.79%**          | 77.53%              | **85.22%**          |
| **Validation Precision** | 87.85%              | 77.51%              | **84.83%**          |
| **Validation Recall**    | 85.98%              | 78.08%              | **85.99%**          |
| **Validation Loss**      | 0.8504              | 0.7545              | **0.6342**          |
| **Train Accuracy**       | 97.89%              | 84.41%              | **96.18%**          |
| **Train Loss**           | 0.0846              | 0.3909              | **0.1200**          |
| **Overfitting Gap**      | 11.35%              | **7.46%**           | 11.32%              |

---

##  **Interpretation**

* The **best score overall** was achieved in **Study 0**, with an F1 of **86.79%**.
* However, **Study 2** produced **more consistent and stable results**, with less variance across trials.
* The **Validation Loss dropped significantly** in Study 2 (from 0.75 → 0.63), which reflects improved learning dynamics.
* The **Overfitting Gap** slightly increased, but this is expected when pushing for higher accuracy.

 In summary:
**Study 2 did not exceed the top score of Study 0, but it successfully converged near the optimum in a stable and reliable manner.**
We now take it as our new starting point for further enhancements.

---

## **Segment-Level Analysis: Average F1 per Class**

|                          | **Best in Study 0** | **Best in Study 2** | **Average Study 0** | **Average Study 2** |
| ------------------------ | ------------------- | ------------------- | ------------------- | ------------------- |
| **Validation Accuracy**  | **86.54%**          | 86.39%              | 76.95%              | **84.86%**          |
| **Validation F1**        | **86.79%**          | 86.72%              | 77.53%              | **85.22%**          |
| **Validation Precision** | **87.85%**          | 86.80%              | 77.51%              | **84.83%**          |
| **Validation Recall**    | 85.98%              | **86.73%**          | 78.08%              | **85.99%**          |
| **Validation Loss**      | 0.8504              | **0.7440**          | 0.7545              | **0.6342**          |
| **Train Accuracy**       | **97.89%**          | 97.68%              | 84.41%              | **96.18%**          |
| **Train Loss**           | 0.0846              | 0.0895              | 0.3909              | **0.1200**          |
| **Overfitting Gap**      | 11.35%              | **11.28%**          | **7.46%**           | 11.32%              |

---

Although the **best single validation score** was achieved in **Study 0** (with a Validation F1 of **86.79%**), the results across trials were inconsistent and highly variable. In contrast, **Study 2** produced **consistently strong results**, with most trials converging around **85% F1** and demonstrating improved loss values and training stability.

In this context, we emphasize the use of **F1 Score** as the main evaluation metric, since it captures both **precision and recall** and is especially informative for **imbalanced datasets** like ours. While accuracy alone may be dominated by the majority classes, the F1 score reflects the model’s true ability to correctly identify all sentiment categories, including the rare or ambiguous ones.

With a reliable baseline now established through Study 2, we turn our attention to **class-level analysis** to identify whether specific segments are underperforming. This will help us target future improvements — such as data augmentation or weighting — more effectively.

---

### **Per-Class Performance Summary**

| **Class**          | **F1 Score** | **Precision** | **Recall**  |
| ------------------ | ------------ | ------------- | ----------- |
| Extremely Positive | **0.82**     | 0.85          | **0.80** 🔽 |
| Neutral            | **0.84**     | 0.86          | **0.81** 🔽 |
| Extremely Negative | 0.86         | **0.82** 🔽   | 0.90        |
| Negative           | 0.87         | 0.87          | 0.88        |
| Positive           | 0.87         | 0.84          | 0.91        |

---

###  **Interpretation:**

* The **weakest classes** based on F1 are:

  * **Extremely Positive (F1 = 0.82)**
  * **Neutral (F1 = 0.84)**

* These classes also have the **lowest Recall** (below 82%), indicating the model **often misses true examples** from these classes.

* For **Extremely Negative**, recall is high (0.90), but **precision is slightly lower** – indicating some false positives.

* **Positive** and **Negative** classes are well-balanced and perform strongly across all metrics.

**Conclusion**:
To improve the model, we should **focus on boosting recall for the Extremely Positive and Neutral classes**, likely through data augmentation, reweighting, or improved representation learning for those categories.

# Augmentations:

#### GUIDED AUGMENTATION PLAN (Based on Study 0 F1 scores):
| Class              | Recommendation            |
|--------------------|---------------------------|
| Negative           | No augmentation           |
| Positive           | No augmentation           |
| Extremely Negative | No augmentation         |
| Neutral            | Add ~2000 samples         |
| Extremely Positive | Add ~2200 samples         |
➤ Focus augmentation only on weaker classes — skip Negative & Positive.




# **STUDY 3:**

**Data Augmentation**:

In [ ]:
# Count and print label distribution before augmentation
print("Label counts BEFORE augmentation:")
original_counts = train_df["Sentiment"].value_counts()
print(original_counts)

# Load full augmented data
augmented_df = pd.read_csv('/content/drive/MyDrive/deep_learning/augmented_1.csv', encoding='latin1')

# Filter only the weak classes
target_classes = ['Extremely Positive', 'Neutral']
augmented_df = augmented_df[augmented_df['Sentiment'].isin(target_classes)]

# Map labels to numeric (assuming label2id is defined)
augmented_df["label"] = augmented_df["Sentiment"].map(label2id)

# Optional: Drop duplicates (based on ProcessedTweet) before merge
before_merge_aug_size = len(augmented_df)
augmented_df = augmented_df.drop_duplicates(subset='ProcessedTweet')
after_merge_aug_size = len(augmented_df)

print(f"\nRemoved {before_merge_aug_size - after_merge_aug_size} duplicate rows from augmented data.")

# Track how many samples were added per class
added_counts = augmented_df["Sentiment"].value_counts()
print("\nNumber of augmented samples added per class:")
print(added_counts)

# Combine datasets
train_df = pd.concat([train_df, augmented_df], ignore_index=True)

# Shuffle the dataset
train_df = train_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Final label counts after augmentation
print("\nLabel counts AFTER augmentation:")
final_counts = train_df["Sentiment"].value_counts()
print(final_counts)


Label counts BEFORE augmentation:
Sentiment
Positive              9137
Negative              7934
Neutral               6170
Extremely Positive    5299
Extremely Negative    4385
Name: count, dtype: int64

Removed 0 duplicate rows from augmented data.

Number of augmented samples added per class:
Sentiment
Extremely Positive    2200
Neutral               2000
Name: count, dtype: int64

Label counts AFTER augmentation:
Sentiment
Positive              9137
Neutral               8170
Negative              7934
Extremely Positive    7499
Extremely Negative    4385
Name: count, dtype: int64


In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 3:")
study3 = optuna.create_study(direction="maximize")
study3.optimize(objective, n_trials=20)


# ============ RUN THE STUDY ============
best_trial = study3.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model3.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")



[I 2025-08-04 06:23:24,396] A new study created in memory with name: no-name-19def74d-fa48-4ffd-9578-2a272bd3ed62


Starting Study 3:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 15/15: Train Acc: 0.9678, Val Acc: 0.8596, Val F1: 0.8630, Gap: 0.1082


Class_Extremely Negative_F1,▁▃▅▄▆█▆██████▇█
Class_Extremely Negative_Precision,▁██▁▄▇▃▇█▅▇▆▆▅█
Class_Extremely Negative_Recall,▅▁▂█▇▅█▅▄▆▆▆▆▇▅
Class_Extremely Positive_F1,▁▅▇▅▆█▆██▇█▇▇▇▇
Class_Extremely Positive_Precision,▁▄▃▅▄▆▆▆▇█▇██▆▅
Class_Extremely Positive_Recall,▁▅█▃▅▆▄▆▆▄▅▄▃▅▇
Class_Negative_F1,▁▅▆▆▅▅▇█▆▆▇▅▄▆▅
Class_Negative_Precision,▄▆▇█▆█▅▇▄▄▇▅▁▅▅
Class_Negative_Recall,▁▄▃▂▃▁▆▅▆▆▄▄█▆▄
Class_Neutral_F1,▁▅▇▇▆▇▇███▇██▇█
Class_Neutral_Precision,▁▁▅▆█▄█▄▅▆▄▆▇▅▆


[I 2025-08-04 22:55:12,222] Trial 19 finished with value: 0.8641885325558795 and parameters: {'learning_rate': 1.6483782188925505e-05, 'weight_decay': 2.4062536869093416e-05, 'patience': 7, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.4, 'gradient_clip_val': 0.5, 'use_class_weights': False}. Best is trial 11 with value: 0.8724489795918368.



 Best Results:
Validation Accuracy: 0.8724
Best hyperparameters:
  learning_rate: 4.964231176733226e-05
  weight_decay: 3.3543626906404976e-05
  patience: 7
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.4
  gradient_clip_val: 0.5
  use_class_weights: False

Best model saved: best_model_trial_11.pt


---

### **Study 3 Summary – Augmented Training Results**

Following the integration of class-specific augmented data, we achieved stable and improved overall performance:

| **Metric**               | **Mean** | **Best Run** |
| ------------------------ | -------- | ------------ |
| **Validation F1**        | 85.68%   | **87.22%**   |
| **Validation Accuracy**  | 85.32%   | **86.89%**   |
| **Validation Precision** | 85.84%   | 87.89%       |
| **Validation Recall**    | 85.76%   | 87.27%       |
| **Overfitting Gap**      | 9.03%    | 12.49%       |
| **Validation Loss**      | 0.561    | —            |

These results confirm a **strong and consistent improvement** over earlier studies, especially in terms of **Recall** and **F1 stability**, despite adding noisy synthetic data.

---

### **Per-Class Performance Check**

| **Class**              | **F1 Score** | **Precision** | **Recall**  |
| ---------------------- | ------------ | ------------- | ----------- |
| **Extremely Positive** | **0.82**     | 0.85          | **0.80** 🔽 |
| **Neutral**            | **0.84**     | 0.86          | **0.81** 🔽 |
| Extremely Negative     | 0.86         | 0.82 🔽       | 0.90        |
| Negative               | 0.87         | 0.87          | 0.88        |
| Positive               | 0.87         | 0.84          | 0.91        |

---

### Conclusion

* The **overall model improved** across all metrics.
* However, **Recall remains slightly lower** for the previously weak classes:

  * **Neutral** (Recall = 0.81)
  * **Extremely Positive** (Recall = 0.80)
* This indicates that while **augmentation helped**, it **did not fully solve the recall gap**.

---


### **Study 4 – Manual Class Weights for Recall Boost**

> In this experiment, we apply **manual class weights** to directly address the **recall gap** observed in weaker classes (Neutral, Extremely Positive).
> This targeted adjustment allows us to amplify the loss contribution of underperforming labels without introducing new data noise or architectural changes.

We set the weights based on the per-class F1 and recall analysis from Study 3:

```python
manual_weights = torch.tensor([
    1.0,  # Extremely Negative – no recall issue
    1.0,  # Negative – balanced
    1.2,  # Neutral – low recall (0.81)
    1.0,  # Positive – high recall (0.91)
    1.3   # Extremely Positive – lowest recall (0.80)
], dtype=torch.float).to(device)
```
Manual weights are applied with moderate strength (1.2–1.3) to further improve recall on previously underperforming classes, without introducing bias or instability.
This configuration keeps the model architecture and training regime (Hyperparameters) identical to Study 3, isolating the effect of **loss reweighting**.

---

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 4:")
study4 = optuna.create_study(direction="maximize")
study4.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study4.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model4.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")



[I 2025-08-05 06:34:10,142] A new study created in memory with name: no-name-8fcbfe29-5525-4ccb-997c-c0b514f5fde4


Starting Study 4:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

[I 2025-08-05 17:58:46,302] Trial 13 finished with value: 0.8696550048590865 and parameters: {'learning_rate': 3.1094326027944724e-05, 'weight_decay': 3.505302260086934e-05, 'patience': 7, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.4, 'gradient_clip_val': 0.5, 'use_class_weights': False}. Best is trial 1 with value: 0.8732993197278912.


Using standard CrossEntropyLoss


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch  1/15: Train Acc: 0.6910, Val Acc: 0.8123, Val F1: 0.8180, Gap: -0.1214


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



#### PREPROCESSING STRATEGY: WHY WE CHOSE A LIGHTER CLEANING FUNCTION


Previously, we used an aggressive text cleaning function (`normalize_text`) designed to:
 - Remove emojis, repeated punctuation, and special characters
 - Normalize COVID-related hashtags (e.g., #covid2019 → #covid)
 - Split hashtags into multiple words (e.g., #StayHomeStaySafe → #Stay #Home #Stay #Safe)
 - Remove duplicate hashtags and gibberish tokens
 - Fix common HTML entities and encoding artifacts

 While this helped create a very "clean" dataset, it may also have stripped away:
 - Hashtag structure that captures tweet-specific sentiment (e.g., #panicbuying, #heroes)
 - Informal Twitter conventions that are useful in real-world classification

 Since the CardiffNLP RoBERTa model (`cardiffnlp/twitter-roberta-base-sentiment-latest`) was pretrained
 specifically on noisy Twitter data (with emojis, mentions, and slang), we now switch to a lighter
 preprocessing function: `clean_for_cardiffnlp`.

 This new function performs minimal normalization:
 - Replaces mentions with "@user" and links with "http"
 - Normalizes common COVID spelling variants (e.g., corona → covid)
 - Leaves emojis, punctuation, and hashtags largely intact

 Rationale for switching:
- Better alignment with the model's original pretraining data
- Preserves real-world phrasing as it appeared in social media
- Avoids over-sanitizing input that the model may already be robust to

#### **In summary:** we believe this lighter cleaning approach will improve model generalization, especially on sentiment-heavy or emotionally nuanced tweets, and will better **leverage the strengths of the CardiffNLP pretrained tokenizer.**





---



---




In [ ]:
def clean_for_cardiffnlp(text):
    if pd.isnull(text):
        return ""

    tokens = []
    for t in text.split(" "):
        if t.startswith("@") and len(t) > 1:
            tokens.append("@user")
        elif t.startswith("http"):
            tokens.append("http")
        else:
            tokens.append(t)
    text = " ".join(tokens)

    # Normalize common COVID variants to "covid"
    text = re.sub(r"\b(coronaviruspandemic|covid[_\s-]*2019|covid[_\s-]*19|covid2019|coronavirus2019|coronavirus|corona)\b", "covid", text, flags=re.IGNORECASE)

    # Decode HTML entities
    text = html.unescape(text)

    # Normalize whitespace and repeated punctuation (optional)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(\.\s*){2,}", ". ", text)
    text = re.sub(r"([!?]){2,}", r"\1", text)
    text = re.sub(r"(\?\s+){2,}", "?", text)
    text = re.sub(r"(\!\s+){2,}", "!", text)

    return text


In [ ]:
# Apply to train and test
train_df['ProcessedTweet'] = train_df['OriginalTweet'].apply(clean_for_cardiffnlp)
eval_df['ProcessedTweet'] = eval_df['OriginalTweet'].apply(clean_for_cardiffnlp)

In [ ]:
# Display
pd.set_option('display.max_colwidth', None)  # so full text is shown
sample = train_df[['ProcessedTweet','Sentiment','label']].sample(10, random_state=24)
display(sample)

,ProcessedTweet,Sentiment,label
17977,New survey by @user finds that only 2% of #consumers think #brands should pause all #advertising; 49% want #ads to make them feel informed and 37% want ads to make them feel warm/happy #digitalmarketing #covid #consumerbehavior http,Extremely Positive,4
31793,Some #Costco stores say their prohibiting people from returning #toiletpaper. Who returns toilet paper? Probably the same people who think their rectums go into remission? #Trending #covid @user @user #JimmyKimmel #ForReal,Neutral,2
8808,"After increasing the prices of platform tickets, the Indian Railways withdrew most concessional ticket facilities in trains as a precautionary measure to contain the spread of #covid. https://t.co/PmUuJd3GI4",Neutral,2
23027,"@user $Hdelacerda. Oil prices caused a massive slow down and now the covid has caused layoffs, I havenÃÂt worked since the beginning of February just trying to make sure my wife and 2 daughters are taken care of.",Extremely Positive,4
6409,My wife just messaged me to say the supermarket shelves were empty. I know what's for dinner tonight #covid #CoronavirusOutbreak http,Negative,1
28137,"I nominate the global covid Responders for the 2020 Time MagazineÃÂs Person of the Year: (including nurses, paramedics, doctors, hospital admin and cleaning staff, grocery store employees, etc) #CoronavirusHeroes #covid #covid",Neutral,2
22277,"WeÃÂd like to take a moment to thank all of the grocery store personnel, cleaners, maintenance workers, truck drivers and warehouse workers for their perseverance and commitment. We are incredibly grateful to those who are making sacrifices to support the covid fight. http",Extremely Positive,4
21206,This is too much! No fizzy san PelegrÃÂ­no water for my wine spritzer OR Houmous! What in gods name do you expect the middle classes to do! To die like dinosaurs! #covid #StayHome24in48 #stockpilingUK #toiletpaper #CoronaCrisis #covid #covid,Extremely Negative,0
15208,? $CAG $GIS $K $KO Consumer Brands Association CEO on the state of global supply chains amid covid http http,Neutral,2
28598,BREAKING: Govt. has announced good news for shoppers who have been battling to buy essential products at inflated prices. Retailers on the naughty list could face hefty fines or even prison sentences. #Covid19SA #COVID19SouthAfrica #CoronaVirusUpdate http,Negative,1


In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 5:")
study5 = optuna.create_study(direction="maximize")
study5.optimize(objective, n_trials=20)


# ============ RUN THE STUDY ============
best_trial = study5.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model5.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")


[I 2025-08-05 19:06:16,430] A new study created in memory with name: no-name-08664731-cc2f-4a24-82e1-b5ce991c06d2


Starting Study 5:
Using standard CrossEntropyLoss


Epoch  1/15: Train Acc: 0.4478, Val Acc: 0.4947, Val F1: 0.5031, Gap: -0.0469


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch  2/15: Train Acc: 0.5019, Val Acc: 0.5228, Val F1: 0.5343, Gap: -0.0209


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 15/15: Train Acc: 0.9533, Val Acc: 0.8625, Val F1: 0.8656, Gap: 0.0909


Class_Extremely Negative_F1,▁▁▅▆▅▇▇▇▆▅▇▇█▇▇
Class_Extremely Negative_Precision,▂▁▄▄▃▇▆▆▅▃▆█▇▆█
Class_Extremely Negative_Recall,▄▇▄▆█▃▅▆▇█▅▁▄▅▁
Class_Extremely Positive_F1,▁▂▅▆▅█▇█▇▆█████
Class_Extremely Positive_Precision,▁▄▄▅▅▆▆▆▇▆▇▇█▆▆
Class_Extremely Positive_Recall,▁▁▅▆▅█▇▇▆▅▇▇▆▇█
Class_Negative_F1,▁▆▇▇█▇██▇██▇▆▇▇
Class_Negative_Precision,▅▄▇▇█▂▇▇▄█▅▃▁▄▃
Class_Negative_Recall,▁▆▆▆▆█▆▇▇▆▇██▇█
Class_Neutral_F1,▁▅▇▇▇▇▇█▇██████
Class_Neutral_Precision,▁▆▅▇▇█▇▇▇▇▇▇▇█▇


[I 2025-08-06 06:25:14,479] Trial 19 finished with value: 0.8624878522837707 and parameters: {'learning_rate': 1.2719263243273816e-05, 'weight_decay': 4.624614331332635e-05, 'patience': 5, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.2, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 15 with value: 0.8672254616132167.



 Best Results:
Validation Accuracy: 0.8672
Best hyperparameters:
  learning_rate: 1.4197883401156393e-05
  weight_decay: 8.35291387420277e-06
  patience: 5
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.4
  gradient_clip_val: 0.5
  use_class_weights: True

Best model saved: best_model_trial_15.pt


## **Study 5 – Evaluation of Dirty Data**

**Objective:**
To assess whether training on uncleaned (dirty) text data improves recall performance, particularly in the weaker sentiment classes.

### General Performance Summary:

| Metric               | Study 0 (Clean) | Study 5 (Dirty) | Change     |
| -------------------- | --------------- | --------------- | ---------- |
| Validation F1        | 86.79%          | 86.71%          | −0.08%     |
| Validation Precision | 87.85%          | 89.72%          | +1.87%     |
| Validation Recall    | 85.98%          | 83.97%          | −2.01%     |
| Validation Loss      | 0.8504          | 0.7374          | Improved   |
| Overfitting Gap      | 11.35%          | 11.57%          | Slightly ↑ |

**Per-Class Impact (Recall):**

* Neutral: 0.893 → 0.858 (−0.035)
* Extremely Positive: 0.845 → 0.827 (−0.018)
* Extremely Negative: 0.894 → 0.853 (−0.041)

**Interpretation:**
While precision improved across all classes, recall decreased significantly in most. Overall F1 remained nearly unchanged, indicating that using noisy input data shifted the model toward conservative predictions, with fewer false positives but more false negatives. Dirty data did not significantly improve overall performance and introduced a trade-off that may not be beneficial for minority class detection.

---

## **Rationale for Continued Augmentation**

Previous experiments (notably Study 3) showed that class-specific augmentation:

* Improved recall in underperforming classes (e.g., Neutral, Extremely Positive)
* Reduced variation across trials
* Did not outperform the best-case result from Study 0, but improved generalization

**Conclusion:**
While augmentation does not always lead to a new best score, it **consistently enhances model robustness** and recall for weaker classes. Therefore, it remains a valuable component, especially when applied selectively and proportionally to targeted classes.



In [ ]:
# Count and print label distribution before augmentation
print("Label counts BEFORE augmentation:")
original_counts = train_df["Sentiment"].value_counts()
print(original_counts)

# Load full augmented data
augmented_df = pd.read_csv('/content/drive/MyDrive/deep_learning/augmented_light333.csv', encoding='latin1')

# Filter only the weak classes
target_classes = ['Extremely Positive', 'Neutral','Extremely Negative']
augmented_df = augmented_df[augmented_df['Sentiment'].isin(target_classes)]

# Map labels to numeric (assuming label2id is defined)
augmented_df["label"] = augmented_df["Sentiment"].map(label2id)

# Optional: Drop duplicates (based on ProcessedTweet) before merge
before_merge_aug_size = len(augmented_df)
augmented_df = augmented_df.drop_duplicates(subset='ProcessedTweet')
after_merge_aug_size = len(augmented_df)

print(f"\nRemoved {before_merge_aug_size - after_merge_aug_size} duplicate rows from augmented data.")

# Track how many samples were added per class
added_counts = augmented_df["Sentiment"].value_counts()
print("\nNumber of augmented samples added per class:")
print(added_counts)

# Combine datasets
train_df = pd.concat([train_df, augmented_df], ignore_index=True)

# Shuffle the dataset
train_df = train_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Final label counts after augmentation
print("\nLabel counts AFTER augmentation:")
final_counts = train_df["Sentiment"].value_counts()
print(final_counts)


Label counts BEFORE augmentation:
Sentiment
Positive              9137
Neutral               8173
Negative              7934
Extremely Positive    6392
Extremely Negative    5694
Name: count, dtype: int64

Removed 0 duplicate rows from augmented data.

Number of augmented samples added per class:
Sentiment
Extremely Negative    579
Extremely Positive    578
Neutral               543
Name: count, dtype: int64

Label counts AFTER augmentation:
Sentiment
Positive              9137
Neutral               8716
Negative              7934
Extremely Positive    6970
Extremely Negative    6273
Name: count, dtype: int64


In [ ]:
# Display
pd.set_option('display.max_colwidth', None)  # so full text is shown
sample = train_df[['ProcessedTweet','Sentiment','label']].sample(10, random_state=24)
display(sample)

,ProcessedTweet,Sentiment,label
25970,Cheryl Idell Chief Research Officer at Entertainment amp Direct to Consumer shares how social distancing in the wake of covid is affecting usage of digital platforms,Extremely Positive,4
15144,you know you #needtogetoutmore when you bump into your sister-in-law at the supermarket and you're so #thankful that you skip home on cloud 9. #StayHomeSaveLives #isolationblues,Neutral,2
6198,"To keep our consumer-members & employees safe, weÃÂre taking additional precautions to ensure the co-op is prepared to keep the lights on. In response to #covid, weÃÂve activated our emergency response plan. Learn about operational changes & plans: http http",Positive,3
13627,"Companies, Trade Organizations, Movie Industry and Newspapers seek to postpone date for the #California Consumer Privacy Act (#CCPA) due to the new #covid and a lack of clarity on the enforcement rules http http",Negative,1
1967,Dutch prime minister visits supermarket and tries to set hoarders at ease there is so much toilet paper available in the country that we can shit for 10 years,Negative,1
27581,Yoda fights R2 to get TOILET PAPER! Funny edit/dub http <---CLICK HERE #covid #toiletpaper #Yoda #R2D2 #brawl #fight #toiletpaperwars #ToiletPaperPanic #ToiletPaperApocalypse #funny #starwars #COVIDIDIOTS #COVIDIOTS #covid #COVID?19 #lol #haha #stock http,Positive,3
2386,"I just hope that in this panic to buy all of the toilet paper/sanitizer/food, you take care of your animals as well as you plan to take care of yourselves and your families. :( #itsnottheendoftheworld #covid",Extremely Positive,4
11231,"Healthcare workers need N95 masks to protect themselves from covid. It makes me sick to see people try to sell these on various websites for crazy high prices. Donate them to a hospital, you assholes!",Extremely Negative,0
29064,Do you currently have hand sanitizer,Positive,3
37617,#covid could halt the rise in house prices ? #covid #housingmarket @user https://t.co/iTecbrnvrg,Neutral,2


In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 6:")
study6 = optuna.create_study(direction="maximize")
study6.optimize(objective, n_trials=25)


# ============ RUN THE STUDY ============
best_trial = study6.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model6.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")


[I 2025-08-06 19:20:53,577] A new study created in memory with name: no-name-44a5ef35-fc05-42c6-a6b1-44a81c820512


Starting Study 6:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Epoch 11/20: Train Acc: 0.9378, Val Acc: 0.8581, Val F1: 0.8625, Gap: 0.0797


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 12/20: Train Acc: 0.9429, Val Acc: 0.8595, Val F1: 0.8634, Gap: 0.0835


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[W 2025-08-07 09:09:02,760] Trial 13 failed with parameters: {'learning_rate': 9.369850790707184e-06, 'weight_decay': 9.072604585299779e-05, 'patience': 5, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.3, 'gradient_clip_val': 1.0, 'use_class_weights': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-2760107258.py", line 93, in objective
    best_val_accuracy = train_model_with_hyperparams(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-748624337.py", line 37, in train_m

KeyboardInterrupt: 

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 6.1:")
study6 = optuna.create_study(direction="maximize")
study6.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study6.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model6.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")


[I 2025-08-07 09:15:59,219] A new study created in memory with name: no-name-0ca1a817-c9f0-487c-8428-61fdd6763a31


Starting Study 6.1:
Computing class weights...
Computing class weights for 39030 samples
Label distribution: [6273 7934 8716 9137 6970]
Class weights computed:
Extremely Negative (0): 1.244
Extremely Positive (1): 0.984
Negative (2): 0.896
Neutral (3): 0.854
Positive (4): 1.120
Using weighted CrossEntropyLoss


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch  1/12: Train Acc: 0.6950, Val Acc: 0.7235, Val F1: 0.7295, Gap: -0.0286


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch  9/12: Train Acc: 0.9384, Val Acc: 0.8562, Val F1: 0.8605, Gap: 0.0822


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 10/12: Train Acc: 0.9416, Val Acc: 0.8584, Val F1: 0.8628, Gap: 0.0832


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study 6.2:")
study6 = optuna.create_study(direction="maximize")
study6.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study6.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = "drive/MyDrive/deep_learning/best_model6.pt"
model = RobertaWithDropout(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest",
    num_labels=5,
    dropout_rate=0.2
)
model.load_state_dict(torch.load(best_model_path))
torch.save(model.state_dict(), save_path)
print(f"\nBest model saved: {best_model_path}")


[I 2025-08-10 13:15:18,551] A new study created in memory with name: no-name-19f0a79f-cb07-47be-ac74-ea51e05c03f9


Starting Study 6.2:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Using standard CrossEntropyLoss


Epoch  1/12: Train Acc: 0.6703, Val Acc: 0.7771, Val F1: 0.7835, Gap: -0.1068
Epoch  2/12: Train Acc: 0.8114, Val Acc: 0.7953, Val F1: 0.8014, Gap: 0.0161
Epoch  3/12: Train Acc: 0.8641, Val Acc: 0.8335, Val F1: 0.8390, Gap: 0.0306
Epoch  4/12: Train Acc: 0.8944, Val Acc: 0.8310, Val F1: 0.8367, Gap: 0.0633
Epoch  5/12: Train Acc: 0.9178, Val Acc: 0.8390, Val F1: 0.8439, Gap: 0.0788
Epoch  6/12: Train Acc: 0.9338, Val Acc: 0.8345, Val F1: 0.8387, Gap: 0.0993
Epoch  7/12: Train Acc: 0.9444, Val Acc: 0.8466, Val F1: 0.8509, Gap: 0.0978
Epoch  8/12: Train Acc: 0.9533, Val Acc: 0.8514, Val F1: 0.8555, Gap: 0.1019
Epoch  9/12: Train Acc: 0.9609, Val Acc: 0.8507, Val F1: 0.8551, Gap: 0.1102
Epoch 10/12: Train Acc: 0.9661, Val Acc: 0.8475, Val F1: 0.8503, Gap: 0.1186
Epoch 11/12: Train Acc: 0.9683, Val Acc: 0.8467, Val F1: 0.8516, Gap: 0.1216
Epoch 12/12: Train Acc: 0.9742, Val Acc: 0.8512, Val F1: 0.8548, Gap: 0.1230


Class_Extremely Negative_F1,▁▁▅▆▆▅▆██▆█▇
Class_Extremely Negative_Precision,▅▁▄▅▄▃▄█▅▄▆▆
Class_Extremely Negative_Recall,▁█▇▅▇▇█▂▇█▅▄
Class_Extremely Positive_F1,▂▁▅▆▆▆▇█▇▆██
Class_Extremely Positive_Precision,▁▃▄▆▇▆▅▆▆█▇▆
Class_Extremely Positive_Recall,▆▁▅▄▄▅▆█▇▂▆▇
Class_Negative_F1,▁▃▇▇▇▇█▆▆▆▅▆
Class_Negative_Precision,▁█▇▆▇▄▇▂▅▁▃▃
Class_Negative_Recall,▄▁▄▅▆▇▆█▆█▆█
Class_Neutral_F1,▁▃▆▄▅▅▇▇▇█▆▇
Class_Neutral_Precision,▁▅▅▅▅██▇▇▅▅▇


[I 2025-08-10 23:38:24,881] Trial 14 finished with value: 0.8514334305150631 and parameters: {'learning_rate': 1.872077428520276e-05, 'weight_decay': 3.803908202135172e-06, 'patience': 5, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.3, 'gradient_clip_val': 1.0, 'use_class_weights': False}. Best is trial 4 with value: 0.8691690962099126.



 Best Results:
Validation Accuracy: 0.8692
Best hyperparameters:
  learning_rate: 2.08371793648536e-05
  weight_decay: 9.080837423383063e-06
  patience: 5
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.3
  gradient_clip_val: 1.0
  use_class_weights: False

Best model saved: best_model_trial_4.pt


**Per-Class Recall Comparison — Study 5 (Dirty Data) vs. Study 6 (Dirty Data + Augmentations)**

| Class                  | Recall (S5) | Recall (S6) | Δ (S6 - S5) | Interpretation                                                 |
| ---------------------- | ----------- | ----------- | ----------- | -------------------------------------------------------------- |
| Extremely Negative     | 0.853       | 0.857       | +0.004      | Negligible improvement, not directly targeted by augmentations |
| Negative               | 0.894       | 0.903       | +0.009      | Slight gain, likely a side effect of overall training          |
| **Neutral**            | **0.858**   | **0.874**   | **+0.016**  | Clear improvement, aligns with augmentation goal               |
| Positive               | 0.916       | 0.913       | −0.003      | Minor drop, negligible impact on overall performance           |
| **Extremely Positive** | **0.827**   | **0.846**   | **+0.019**  | Clear improvement, aligns with augmentation goal               |

**Summary:**
Compared to Study 5, the augmentations in Study 6 achieved their primary objective — improving recall for the **Neutral** (+1.6%) and **Extremely Positive** (+1.9%) classes, which were previously underperforming. Other classes showed only minor fluctuations, with no significant negative effects on overall performance.
